**Install dependencies**

In [1]:
# %%capture
# !pip install onnx
# !pip install onnxruntime
# !pip install onnx-tf
# !pip install tensorflow
# !pip install -U tensorflowjs
# !pip install simple-onnx-processing-tools
# !pip install nvidia-pyindex
# !pip install onnx-graphsurgeon
# !pip install tensorflow-probability

**Import dependencies**

In [2]:
import os
from glob import glob

from pathlib import Path

# Import pandas module for data manipulation
import pandas as pd

# Set options for Pandas DataFrame display
pd.set_option('max_colwidth', None)  # Do not truncate the contents of cells in the DataFrame
pd.set_option('display.max_rows', None)  # Display all rows in the DataFrame
pd.set_option('display.max_columns', None)  # Display all columns in the DataFrame

import onnx

In [3]:
from scc4onnx import order_conversion

In [4]:
from onnx_tf.backend import prepare

2023-03-11 11:30:38.295081: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 11:30:38.362871: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-11 11:30:38.733446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-11 11:30:38.733482: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

**Define data directory**

In [5]:
# Create a directory to store the checkpoints if it does not already exist
# checkpoint_dir = Path("./miniai_data_augmentation_seed-1/")
# checkpoint_dir = Path("./miniai_data_augmentation_seed-42/")
# checkpoint_dir = Path("./miniai_data_augmentation_seed-100/")
checkpoint_dir = Path("./miniai_data_augmentation_seed-1234/")
checkpoint_dir.mkdir(parents=True, exist_ok=True)

# Print the checkpoint path
checkpoint_dir

PosixPath('miniai_data_augmentation_seed-1234')

**Get available ONNX files**

In [6]:
onnx_paths = [Path(path) for path in glob(os.path.join(checkpoint_dir, f"*.onnx"))]
pd.DataFrame([path.name for path in onnx_paths])

,0
0,resnet18-baseline-opset-12-float32.onnx
1,resnet18-custom-trivial-aug-1-opset-12-float32.onnx
2,resnet18-rand-pixel-cp-1-opset-12-float32.onnx
3,resnet18-rand-square-cp-1-opset-12-float32.onnx
4,resnet18-rand-square-pixel-cp-1-opset-12-float32.onnx
5,resnet18-trivial-aug-item-1-opset-12-float32.onnx
6,resnet18d-custom-trivial-aug-1-opset-12-float32.onnx


**Select ONNX model**

In [7]:
onnx_model_path = onnx_paths[5]
onnx_model_path.name

'resnet18-trivial-aug-item-1-opset-12-float32.onnx'

**Load ONNX model**

In [8]:
onnx_model = onnx.load(onnx_model_path)

**Inspect model input**

In [9]:
onnx_model.graph.input[0]

name: "input.1"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 224
      }
      dim {
        dim_value: 224
      }
    }
  }
}

**Get input name**

In [10]:
input_name = onnx_model.graph.input[0].name
input_name

'input.1'

**Convert model input to channels-last format**

In [11]:
onnx_model = order_conversion(
    onnx_graph=onnx_model,
    input_op_names_and_order_dims={f"{input_name}": [0,2,3,1]},
    non_verbose=True
)

**Inspect updated model input**

In [12]:
onnx_model.graph.input[0]

name: "input.1"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 224
      }
      dim {
        dim_value: 224
      }
      dim {
        dim_value: 3
      }
    }
  }
}

**Prepare ONNX model for Tensorflow Backend**

In [13]:
tf_rep = prepare(onnx_model)

2023-03-11 11:30:39.781458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-11 11:30:39.807116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-11 11:30:39.807238: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-11 11:30:39.808002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-11 11:30:39.808112: I tensorflow/compiler/xla/stream_executo

**Define path for TensorFlow [saved model](https://www.tensorflow.org/guide/saved_model) directory**

In [14]:
tf_model_dir = f"{checkpoint_dir}/{onnx_model_path.name.split('.')[0]}"
tf_model_dir

'miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32'

**Export backend representation to a Tensorflow proto file**

In [15]:
tf_rep.export_graph(tf_model_dir)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


INFO:tensorflow:Assets written to: miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32/assets


INFO:tensorflow:Assets written to: miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32/assets


**Define directory path to store tfjs model files**

In [16]:
tfjs_model_dir = f"{tf_model_dir}-tfjs-uint8"
tfjs_model_dir

'miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32-tfjs-uint8'

**Define arguments for tfjs converter script**

In [17]:
tfjs_convert_command = f"""tensorflowjs_converter
                 --input_format=tf_saved_model 
                 --output_format=tfjs_graph_model 
                 --signature_name=serving_default 
                 --saved_model_tags=serve 
                 "{tf_model_dir}" 
                 "{tfjs_model_dir}"
                 "--quantize_uint8"
                 """
tfjs_convert_command = " ".join(tfjs_convert_command.split())
tfjs_convert_command

'tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default --saved_model_tags=serve "miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32" "miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32-tfjs-uint8" "--quantize_uint8"'

**Export SavedModel to TFJS format**

In [18]:
print("Exporting TensorFlow SavedModel to TensorFlow.js Graph model...")
conversion_result = %sx $tfjs_convert_command
print("\n".join(conversion_result))

Exporting TensorFlow SavedModel to TensorFlow.js Graph model...
2023-03-11 11:30:44.070424: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-11 11:30:44.070458: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-11 11:30:44.070462: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Writing weight file miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32-tfjs-uint8/model.json...


**Archive tfjs folder for download**

In [19]:
!tar vcfz {tfjs_model_dir}.tar.gz {tfjs_model_dir}

miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32-tfjs-uint8/
miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32-tfjs-uint8/group1-shard1of3.bin
miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32-tfjs-uint8/group1-shard2of3.bin
miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32-tfjs-uint8/group1-shard3of3.bin
miniai_data_augmentation_seed-1234/resnet18-trivial-aug-item-1-opset-12-float32-tfjs-uint8/model.json
